In [32]:
import os 
import json
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

In [3]:
load_dotenv(override=True)
client = OpenAI(base_url='https://generativelanguage.googleapis.com/v1beta/openai/',api_key=os.getenv('GEMINI_API_KEY'))

In [4]:
def response_ai(prompt , history):
    history = [{'role':h['role'],'content':h['content']}for h in history]
    message = [{'role':'user','content':prompt}] + history
    response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,stream=True)
    response_text = ''
    for chunk in response:
        response_text += chunk.choices[0].delta.content
        yield response_text
    

In [5]:
gr.ChatInterface(fn=response_ai,type='messages', title="Airport Chatbot", description="Ask me anything about airports!").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [6]:
# creating a simple funcation

ticket_avaliable = {'uk':'12','us':'13','fr':'14'}

def check_ticket(country):
    print(f'Tool call for {country}')
    price = ticket_avaliable.get(country,'Unknow number of tickets')
    return f"The price of this {country} ticket is {price}"



In [7]:
check_ticket('fr')

Tool call for fr


'The price of this fr ticket is 14'

Function for tools

In [8]:
ticket_price = {'domail':'50','chatter':'30','chatter class':'150'}

def get_price_to_univerity(location):
    print(f'Tool called for the location {location}')
    prince = ticket_price.get(location.lower(),'out of reached location')
    return f'The price of give {location} is {prince}'


In [12]:
get_price_to_univerity('chatter class')

Tool called for the location chatter class


'The price of give chatter class is 150'

In [13]:
#This dictinoary structure is used to described the funcation
price_loaction ={
    'name': 'get_price_to_univerity',
    'description': 'they return the price of ticket for the given loaction',
    'parameters': {
        'type':'object',
        'properties':{
            'location': {
                'type':'string',
                'description':'The location for which the price is to be retrieved'
            },
        },
        'required':['location'],
        'additionalProperties':False

    }
}

In [14]:
#include this funcation to tools
tools = [{'type':'function','function':price_loaction}]

In [15]:
tools

[{'type': 'function',
  'function': {'name': 'get_price_to_univerity',
   'description': 'they return the price of ticket for the given loaction',
   'parameters': {'type': 'object',
    'properties': {'location': {'type': 'string',
      'description': 'The location for which the price is to be retrieved'}},
    'required': ['location'],
    'additionalProperties': False}}}]

In [36]:
system_message = """ Your a helpfull assitent your tasked is to understand the prompt of
user and from it get the name of location from there he/she went to go and returned
the price of that location
"""

Now Passing the tools to the gemini model and see the response

In [ ]:
def chat(message,history):
    history = [{'role':h['role'],'content':h['content']}for h in history]
    messages = [{'role':'system','content':'system_message'}]+ [{'role':'user','content':message}] + history
    response = client.chat.completions.create(model='gemini-2.5-flash',messages=messages,tools=tools)

    if response.choices[0].finish_reason == 'tool_call':
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = client.chat.completions.create(model='gemini-2.5-flash',messages=messages)

    return response.choices[0].message.content    


In [38]:
#We have to define the funcation to handle the tool call
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == 'get_price_to_university':
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('location')
        price_details = get_price_to_univerity(city)
        response = {
            'role':'tool',
            'content': price_details,
            'tool_call_id': tool_call.id
        }
    return response    

In [39]:
gr.ChatInterface(fn=chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
